In [1]:
# Trained tensorflow model

model_file_name = r"SagittalSpine_04.h5"
models_folder_name = r"SavedModels"

# Input ultrasound sequence names

input_browser_name = r"SagittalScan"
input_image_name = r"Image_Image"

# Output will be saved using these names

output_browser_name = r"BoneSequenceBrowser"
output_sequence_name = r"SegmentationSequence"
output_image_name = r"Segmented_Image"

# Optionally save output to numpy arrays

array_output = True
array_folder_name = r"Temp"
array_segmentation_name = r"segmentation"
array_ultrasound_name = r"ultrasound"

# Image processing parameters

# Erases the side of prediction images. 1.0 means the whole prediction is erased.
# Background should be the first component (i.e. y[:,:,:,0]) in the prediction output array.

clip_side_ratio = 0.3
apply_logarithmic_transformation = True
logarithmic_transformation_decimals = 4

In [2]:
import datetime
import numpy as np
import os
import scipy.ndimage

from keras.models import load_model

from local_vars import root_folder

In [3]:
# Check if keras model file exists. Abort if not found. Load model otherwise.

models_path = os.path.join(root_folder, models_folder_name)
model_fullpath = os.path.join(models_path, model_file_name)

if not os.path.exists(model_fullpath):
    raise Exception("Could not find model: " + model_fullpath)

print("Loading model from: " + model_fullpath)

if array_output:
    array_output_fullpath = os.path.join(root_folder, array_folder_name)
    array_segmentation_fullname = os.path.join(array_output_fullpath, array_segmentation_name)
    array_ultrasound_fullname = os.path.join(array_output_fullpath, array_ultrasound_name)
    if not os.path.exists(array_output_fullpath):
        os.mkdir(array_output_fullpath)
        print("Folder created: {}".format(array_output_fullpath))
    print("Will save segmentation output to {}".format(array_segmentation_fullname))
    print("Will save ultrasound output to   {}".format(array_ultrasound_fullname))


from keras import backend as K

def dice_coef(y_true, y_pred, smooth=1):
    """
    Dice = (2*|X & Y|)/ (|X|+ |Y|)
         =  2*sum(|A*B|)/(sum(A^2)+sum(B^2))
    ref: https://arxiv.org/pdf/1606.04797v1.pdf
    """
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    
    return (2. * intersection + smooth) / (K.sum(K.square(y_true),-1) + K.sum(K.square(y_pred),-1) + smooth)


def weighted_categorical_crossentropy(weights):
    """
    A weighted version of keras.objectives.categorical_crossentropy

    Variables:
        weights: numpy array of shape (C,) where C is the number of classes
    """
    weights = K.variable(weights)

    def loss(y_true, y_pred):
        # scale predictions so that the class probas of each sample sum to 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        # clip to prevent NaN's and Inf's
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        # calc
        loss = y_true * K.log(y_pred) * weights
        loss = -K.sum(loss, -1)

        return loss

    return loss

wcce = weighted_categorical_crossentropy([0.05, 0.95])

model = load_model(model_fullpath, custom_objects={'loss': wcce, 'dice_coef': dice_coef})

# model.summary()

Loading model from: d:\Data\SavedModels\SagittalSpine_04.h5
Will save segmentation output to d:\Data\Temp\segmentation
Will save ultrasound output to   d:\Data\Temp\ultrasound


In [4]:
# Check input. Abort if browser or image doesn't exist.

input_browser_node = slicer.util.getFirstNodeByName(input_browser_name, className='vtkMRMLSequenceBrowserNode')
input_image_node = slicer.util.getFirstNodeByName(input_image_name, className="vtkMRMLScalarVolumeNode")

if input_browser_node is None:
    logging.error("Could not find input browser node: {}".format(input_browser_node))
    raise

if input_image_node is None:
    logging.error("Could not find input image node: {}".format(input_image_name))
    raise

In [5]:
# Create output image and browser for segmentation output.

output_browser_node = slicer.util.getFirstNodeByName(output_browser_name, className='vtkMRMLSequenceBrowserNode')
if output_browser_node is None:
    output_browser_node = slicer.mrmlScene.AddNewNodeByClass('vtkMRMLSequenceBrowserNode', output_browser_name)

output_sequence_node = slicer.util.getFirstNodeByName(output_sequence_name, className="vtkMRMLSequenceNode")
if output_sequence_node is None:
    output_sequence_node = slicer.mrmlScene.AddNewNodeByClass('vtkMRMLSequenceNode', output_sequence_name)
    output_browser_node.AddSynchronizedSequenceNode(output_sequence_node)

output_image_node = slicer.util.getFirstNodeByName(output_image_name, className="vtkMRMLScalarVolumeNode")
if output_image_node is None:
    volumes_logic = slicer.modules.volumes.logic()
    output_image_node = volumes_logic.CloneVolume(slicer.mrmlScene, input_image_node, output_image_name)
    browser_logic = slicer.modules.sequencebrowser.logic()
    browser_logic.AddSynchronizedNode(output_sequence_node, output_image_node, output_browser_node)

output_browser_node.SetRecording(output_sequence_node, True)

In [6]:
# Add all input sequences to the output browser for being able to conveniently replay everything

proxy_collection = vtk.vtkCollection()
input_browser_node.GetAllProxyNodes(proxy_collection)

for i in range(proxy_collection.GetNumberOfItems()):
    proxy_node = proxy_collection.GetItemAsObject(i)
    output_sequence = slicer.mrmlScene.AddNewNodeByClass('vtkMRMLSequenceNode')
    browser_logic.AddSynchronizedNode(output_sequence, proxy_node, output_browser_node)
    output_browser_node.SetRecording(output_sequence, True)

In [7]:
# Iterate input sequence, compute segmentation for each frame, record output sequence.

num_items = input_browser_node.GetNumberOfItems()
n = num_items
input_browser_node.SelectFirstItem()

input_array = slicer.util.array(input_image_node.GetID())
slicer_to_model_scaling = model.layers[0].input_shape[1] / input_array.shape[1]
model_to_slicer_scaling = input_array.shape[1] / model.layers[0].input_shape[1]

print("Will segment {} images".format(n))

if array_output:
    array_output_ultrasound = np.zeros((n, input_array.shape[1], input_array.shape[1]))
    array_output_segmentation = np.zeros((n, input_array.shape[1], input_array.shape[1]), dtype=np.uint8)

Will segment 3396 images


In [8]:
model_output_size = model.layers[-1].output_shape[1]
num_output_components = model.layers[-1].output_shape[3]

mask_model = np.ones([model_output_size, model_output_size])
mask_model_background = np.zeros([model_output_size, model_output_size])

columns_to_mask = int(model_output_size / 2 * clip_side_ratio)
print("Will mask {} columns on both sides".format(columns_to_mask))

mask_model[:,:columns_to_mask] = 0
mask_model[:,-columns_to_mask:] = 0
mask_model_background[:,:columns_to_mask] = 1
mask_model_background[:,-columns_to_mask:] = 1

# Display mask

# import matplotlib
# matplotlib.use('WXAgg')

# from matplotlib import pyplot as plt

# plt.imshow(mask_model[:,:])
# plt.show()

Will mask 19 columns on both sides


In [9]:
start_timestamp = datetime.datetime.now()
print("Processing started at: {}".format(start_timestamp.strftime('%H-%M-%S')))


for i in range(n):
    # if i > 10:  # todo Just for debugging
    #     break
    input_array = slicer.util.array(input_image_node.GetID())
    
    if array_output:
        array_output_ultrasound[i, :, :] = input_array[0, :, :]
    
    resized_input_array = scipy.ndimage.zoom(input_array[0,:,:], slicer_to_model_scaling)
    resized_input_array = np.flip(resized_input_array, axis=0)
    resized_input_array = resized_input_array / resized_input_array.max()  # Scaling intensity to 0-1
    resized_input_array = np.expand_dims(resized_input_array, axis=0)
    resized_input_array = np.expand_dims(resized_input_array, axis=3)
    y = model.predict(resized_input_array)
    if apply_logarithmic_transformation:
        e = logarithmic_transformation_decimals
        y = np.log10(np.clip(y, 10**(-e), 1.0)*(10**e))/e
    y[0,:,:,:] = np.flip(y[0,:,:,:], axis=0)
    
    for component in range(1, num_output_components):
        y[0,:,:,component] = y[0,:,:,component] * mask_model[:,:]
    y[0,:,:,0] = np.maximum(y[0,:,:,0], mask_model_background)
    
    upscaled_output_array = scipy.ndimage.zoom(y[0,:,:,1], model_to_slicer_scaling)
    upscaled_output_array = upscaled_output_array * 255
    upscaled_output_array = np.clip(upscaled_output_array, 0, 255)
    
    if array_output:
        array_output_segmentation[i, :, :] = upscaled_output_array[:, :].astype(np.uint8)
    
    # output_array = slicer.util.array(output_image_node.GetID())
    # output_array[0, :, :] = upscaled_output_array[:, :].astype(np.uint8)
    
    slicer.util.updateVolumeFromArray(output_image_node, upscaled_output_array.astype(np.uint8)[np.newaxis, ...])
    
    output_browser_node.SaveProxyNodesState()
    input_browser_node.SelectNextItem()
    
    slicer.app.processEvents()
    # print("Processed frame {:02d} at {}".format(i, datetime.datetime.now().strftime('%H-%M-%S')))


stop_timestamp = datetime.datetime.now()
print("Processing finished at: {}".format(stop_timestamp.strftime('%H-%M-%S')))


if array_output:
    np.save(array_ultrasound_fullname, array_output_ultrasound)
    np.save(array_segmentation_fullname, array_output_segmentation)
    print("Saved {}".format(array_ultrasound_fullname))
    print("Saved {}".format(array_segmentation_fullname))

Processing started at: 11-37-50
Processing finished at: 11-45-00
Saved d:\Data\Temp\ultrasound
Saved d:\Data\Temp\segmentation


In [10]:
time_seconds = (stop_timestamp - start_timestamp).total_seconds()
print("Processed {} frames in {:.2f} seconds".format(n, time_seconds))
print("FPS = {:.2f}".format(n / time_seconds))

Processed 3396 frames in 430.81 seconds
FPS = 7.88
